# 留学生去留决策量表 V2 — 数据分析报告

**数据来源**: `decision_matrix_v2.db` (SQLite)  
**分析日期**: 2026-02-16  
**版本**: V2（29题，5档中性选项）

---

## 目录
1. 数据加载与预处理
2. 总体概览
3. 象限人群分布分析
4. 个人背景画像（性别 / 城市 / 来美年数 / 年龄）
5. 关注点分析：权重偏好与核心诉求
6. 维度差异化分析：方差最大的题目
7. 分群对比：留美派 vs 回国派的核心差异
8. 美国得分 vs 中国得分 散点象限图
9. 总结

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# ---------- 中文字体配置 ----------
import os

# 清除 matplotlib 字体缓存并注册中文字体
FONT_PATH = '/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
assert os.path.exists(FONT_PATH), f"Chinese font not found at {FONT_PATH}"

fm.fontManager.addfont(FONT_PATH)
zh_font = fm.FontProperties(fname=FONT_PATH)
font_name = zh_font.get_name()

# 全局设置
plt.rcParams.update({
    'font.sans-serif': [font_name, 'DejaVu Sans'],
    'font.family': 'sans-serif',
    'axes.unicode_minus': False,
    'figure.dpi': 150,
    'savefig.dpi': 150,
    'figure.facecolor': 'white',
})

sns.set_theme(style='whitegrid', rc={
    'font.sans-serif': [font_name, 'DejaVu Sans'],
    'axes.unicode_minus': False,
})

# 输出目录
IMG_DIR = 'charts'
os.makedirs(IMG_DIR, exist_ok=True)

print(f'中文字体: {font_name} ({FONT_PATH})')
print('图表输出目录:', os.path.abspath(IMG_DIR))

---
## 1. 数据加载与预处理

In [ ]:
DB_PATH = '../backend/decision_matrix_v2.db'
conn = sqlite3.connect(DB_PATH)

sessions = pd.read_sql('SELECT * FROM sessions', conn)
answers = pd.read_sql('SELECT * FROM answers', conn)
conn.close()

sessions['created_at'] = pd.to_datetime(sessions['created_at'])
print(f'总提交数: {len(sessions)}')
print(f'总答题记录: {len(answers)}')
print(f'数据时间范围: {sessions["created_at"].min()} ~ {sessions["created_at"].max()}')
sessions.head()

In [ ]:
# ---------- 题目元数据 ----------
QUESTION_TITLES = {
    0: '性别', 1: '城市级别', 2: '来美年数', 3: '年龄',
    4: '薪资购买力', 5: '住房压力', 6: '工作强度(WLB)',
    7: '带薪年假(PTO)', 8: '职场年龄压力', 9: '裁员风险',
    10: '副业可能性', 11: '创业环境', 12: '行业天花板',
    13: '身份状态', 14: '落户门槛', 15: '文娱共鸣',
    16: '旅行偏好', 17: '饮食偏好', 18: '出行方式',
    19: '社交边界感', 20: '少数群体权利', 21: '性别角色压力',
    22: '家庭照料义务', 23: '父母财务支持', 24: '父母施压',
    25: '国内人脉', 26: '伴侣适配', 27: '择偶池', 28: '下一代教育',
}

CATEGORIES = {
    '基础信息': [0, 1, 2, 3],
    '职业与财务硬件': [4, 5, 6, 7, 8, 9, 10, 11, 12],
    '身份与生活方式': [13, 14, 15, 16, 17, 18, 19, 20, 21],
    '家庭、情感与社会资本': [22, 23, 24, 25, 26, 27, 28],
}

QUADRANT_LABELS = {
    'us_high_cn_low': '坚定留美派',
    'us_low_cn_high': '果断回国派',
    'us_high_cn_high': '跨国撕裂型',
    'us_low_cn_low': '两难探索型',
}

QUADRANT_COLORS = {
    'us_high_cn_low': '#667eea',
    'us_low_cn_high': '#f5576c',
    'us_high_cn_high': '#43e97b',
    'us_low_cn_low': '#fa709a',
}

sessions['quadrant_label'] = sessions['quadrant'].map(QUADRANT_LABELS)

# ---------- 选项标签映射 ----------
OPTION_LABELS = {
    0: {'A': '女性', 'B': '男性', 'C': '非二元/不透露'},
    1: {'A': '超一线', 'B': '一线/强二线', 'C': '普通二线', 'D': '三线及以下', 'E': '海外长居'},
    2: {'A': '<1年', 'B': '1-3年', 'C': '3-5年', 'D': '5-10年', 'E': '10年+'},
    3: {'A': '20-22岁', 'B': '23-25岁', 'C': '26-28岁', 'D': '29-31岁', 'E': '32岁+'},
}

# 分数映射
OPTION_SCORES = {
    0: {'A': (4,2), 'B': (3,3), 'C': (5,1)},
    1: {'A': (2,5), 'B': (3,4), 'C': (3,3), 'D': (4,2), 'E': (5,1)},
    2: {'A': (2,4), 'B': (3,3), 'C': (4,2), 'D': (5,1), 'E': (5,1)},
    3: {'A': (1,5), 'B': (2,4), 'C': (3,3), 'D': (4,2), 'E': (5,1)},
    13: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
    14: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
    15: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
    16: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
    17: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
    18: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
    19: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
    20: {'A': (5,1), 'B': (4,2), 'C': (3,3)},
    21: {'A': (5,1), 'B': (4,2), 'C': (3,3)},
    22: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
    23: {'A': (3,3), 'B': (3,3), 'C': (2,4), 'D': (1,5)},
    24: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
    25: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
    26: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
    27: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
    28: {'A': (5,1), 'B': (4,2), 'C': (3,3), 'D': (2,4), 'E': (1,5)},
}

WEIGHT_MULTIPLIER = {1: 1.0, 2: 1.25, 3: 1.5, 4: 1.75, 5: 2.0}
TIER3_TO_SCORE = {1: 1, 2: 3, 3: 5}
DUAL_Q_5TIER = [4, 5, 6, 7, 8]  # 5档
DUAL_Q_3TIER = [9, 10, 11, 12]   # 3档

print('元数据加载完成 ✓')

In [ ]:
# ---------- 解析每条答题的 US / CN base_score ----------
def parse_answer(row):
    """为每条 answer 解析出 us_base, cn_base"""
    qid = row['question_id']
    opt = row['selected_option']
    
    if qid in range(4, 13):  # dual_select
        parts = opt.split(',')
        us_tier = int(parts[0].split(':')[1])
        cn_tier = int(parts[1].split(':')[1])
        if qid in DUAL_Q_5TIER:
            return us_tier, cn_tier
        else:  # 3tier
            return TIER3_TO_SCORE[us_tier], TIER3_TO_SCORE[cn_tier]
    else:  # single_choice
        scores = OPTION_SCORES.get(qid, {}).get(opt, (3, 3))
        return scores[0], scores[1]

parsed = answers.apply(parse_answer, axis=1, result_type='expand')
answers['us_base'] = parsed[0]
answers['cn_base'] = parsed[1]
answers['multiplier'] = answers['weight'].map(WEIGHT_MULTIPLIER)
answers['us_weighted'] = answers['us_base'] * answers['multiplier']
answers['cn_weighted'] = answers['cn_base'] * answers['multiplier']
answers['title'] = answers['question_id'].map(QUESTION_TITLES)

# 合并 quadrant 到 answers
answers = answers.merge(sessions[['id', 'quadrant', 'quadrant_label']], 
                         left_on='session_id', right_on='id', suffixes=('', '_sess'))

print(f'答题数据解析完成: {len(answers)} 条')
answers[['session_id', 'question_id', 'title', 'selected_option', 'weight', 
         'us_base', 'cn_base', 'us_weighted', 'cn_weighted']].head(10)

---
## 2. 总体概览

In [ ]:
print(f"{'='*50}")
print(f"  留学生去留决策量表 V2 — 数据总览")
print(f"{'='*50}")
print(f"  总提交人数:   {len(sessions)}")
print(f"  平均美国得分: {sessions['us_total_score'].mean():.1f}  (中位数 {sessions['us_total_score'].median():.1f})")
print(f"  平均中国得分: {sessions['cn_total_score'].mean():.1f}  (中位数 {sessions['cn_total_score'].median():.1f})")
print(f"  中性阈值:     130.5")
print(f"  得分范围(US): {sessions['us_total_score'].min():.1f} ~ {sessions['us_total_score'].max():.1f}")
print(f"  得分范围(CN): {sessions['cn_total_score'].min():.1f} ~ {sessions['cn_total_score'].max():.1f}")
print(f"{'='*50}")

---
## 3. 象限人群分布分析

In [ ]:
# 象限分布 — 饼图 + 条形图
quadrant_counts = sessions['quadrant'].value_counts()
labels_order = ['us_high_cn_low', 'us_low_cn_high', 'us_high_cn_high', 'us_low_cn_low']
quadrant_counts = quadrant_counts.reindex(labels_order)

display_labels = [QUADRANT_LABELS[q] for q in labels_order]
colors = [QUADRANT_COLORS[q] for q in labels_order]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# 饼图
wedges, texts, autotexts = ax1.pie(
    quadrant_counts.values, labels=display_labels, colors=colors,
    autopct=lambda pct: f'{pct:.1f}%\n({int(round(pct/100*len(sessions)))}人)',
    startangle=90, textprops={'fontproperties': zh_font, 'fontsize': 11}
)
for at in autotexts:
    at.set_fontproperties(zh_font)
    at.set_fontsize(9)
ax1.set_title('象限人群分布 — 饼图', fontproperties=zh_font, fontsize=14, fontweight='bold')

# 条形图
bars = ax2.barh(display_labels[::-1], quadrant_counts.values[::-1], color=colors[::-1], edgecolor='white')
for bar, val in zip(bars, quadrant_counts.values[::-1]):
    ax2.text(bar.get_width() + 10, bar.get_y() + bar.get_height()/2, 
             f'{val} ({val/len(sessions)*100:.1f}%)', va='center', fontproperties=zh_font, fontsize=11)
ax2.set_xlabel('人数', fontproperties=zh_font, fontsize=12)
ax2.set_title('象限人群分布 — 条形图', fontproperties=zh_font, fontsize=14, fontweight='bold')
ax2.set_yticklabels(display_labels[::-1], fontproperties=zh_font)
ax2.set_xlim(0, max(quadrant_counts.values) * 1.3)

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/01_quadrant_distribution.png', bbox_inches='tight')
plt.show()

print('\n分析说明:')
dominant = quadrant_counts.idxmax()
print(f'  最大群体: {QUADRANT_LABELS[dominant]} ({quadrant_counts[dominant]}人, {quadrant_counts[dominant]/len(sessions)*100:.1f}%)')
print(f'  最小群体: {QUADRANT_LABELS[quadrant_counts.idxmin()]} ({quadrant_counts.min()}人, {quadrant_counts.min()/len(sessions)*100:.1f}%)')

---
## 4. 个人背景画像

In [ ]:
# ---------- 4.1 性别分布 ----------
gender_data = answers[answers['question_id'] == 0].copy()
gender_data['label'] = gender_data['selected_option'].map(OPTION_LABELS[0])
gender_counts = gender_data['label'].value_counts()

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 性别
ax = axes[0, 0]
gender_colors = ['#ff6b9d', '#4ecdc4', '#ffe66d']
wedges, texts, autotexts = ax.pie(
    gender_counts.values, labels=gender_counts.index, colors=gender_colors[:len(gender_counts)],
    autopct=lambda pct: f'{pct:.1f}%\n({int(round(pct/100*len(sessions)))}人)',
    startangle=90, textprops={'fontproperties': zh_font, 'fontsize': 10}
)
for at in autotexts:
    at.set_fontproperties(zh_font)
    at.set_fontsize(9)
ax.set_title('4.1 性别分布', fontproperties=zh_font, fontsize=13, fontweight='bold')

# ---------- 4.2 城市级别 ----------
city_data = answers[answers['question_id'] == 1].copy()
city_data['label'] = city_data['selected_option'].map(OPTION_LABELS[1])
city_order = ['超一线', '一线/强二线', '普通二线', '三线及以下', '海外长居']
city_counts = city_data['label'].value_counts().reindex(city_order).fillna(0).astype(int)

ax = axes[0, 1]
city_colors = sns.color_palette('YlOrRd', len(city_order))
bars = ax.bar(range(len(city_order)), city_counts.values, color=city_colors, edgecolor='white')
ax.set_xticks(range(len(city_order)))
ax.set_xticklabels(city_order, fontproperties=zh_font, fontsize=9)
for bar, val in zip(bars, city_counts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
            f'{val}\n({val/len(sessions)*100:.1f}%)', ha='center', fontproperties=zh_font, fontsize=9)
ax.set_ylabel('人数', fontproperties=zh_font)
ax.set_title('4.2 家庭城市级别分布', fontproperties=zh_font, fontsize=13, fontweight='bold')

# ---------- 4.3 来美年数 ----------
years_data = answers[answers['question_id'] == 2].copy()
years_data['label'] = years_data['selected_option'].map(OPTION_LABELS[2])
years_order = ['<1年', '1-3年', '3-5年', '5-10年', '10年+']
years_counts = years_data['label'].value_counts().reindex(years_order).fillna(0).astype(int)

ax = axes[1, 0]
yr_colors = sns.color_palette('Blues_d', len(years_order))
bars = ax.bar(range(len(years_order)), years_counts.values, color=yr_colors, edgecolor='white')
ax.set_xticks(range(len(years_order)))
ax.set_xticklabels(years_order, fontproperties=zh_font, fontsize=10)
for bar, val in zip(bars, years_counts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
            f'{val}\n({val/len(sessions)*100:.1f}%)', ha='center', fontproperties=zh_font, fontsize=9)
ax.set_ylabel('人数', fontproperties=zh_font)
ax.set_title('4.3 来美年数分布', fontproperties=zh_font, fontsize=13, fontweight='bold')

# ---------- 4.4 年龄分布 ----------
age_data = answers[answers['question_id'] == 3].copy()
age_data['label'] = age_data['selected_option'].map(OPTION_LABELS[3])
age_order = ['20-22岁', '23-25岁', '26-28岁', '29-31岁', '32岁+']
age_counts = age_data['label'].value_counts().reindex(age_order).fillna(0).astype(int)

ax = axes[1, 1]
age_colors = sns.color_palette('Greens_d', len(age_order))
bars = ax.bar(range(len(age_order)), age_counts.values, color=age_colors, edgecolor='white')
ax.set_xticks(range(len(age_order)))
ax.set_xticklabels(age_order, fontproperties=zh_font, fontsize=10)
for bar, val in zip(bars, age_counts.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
            f'{val}\n({val/len(sessions)*100:.1f}%)', ha='center', fontproperties=zh_font, fontsize=9)
ax.set_ylabel('人数', fontproperties=zh_font)
ax.set_title('4.4 年龄分布', fontproperties=zh_font, fontsize=13, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/02_demographics.png', bbox_inches='tight')
plt.show()

# 统计摘要
age_midpoints = {'A': 21, 'B': 24, 'C': 27, 'D': 30, 'E': 33}
age_raw = answers[answers['question_id'] == 3]['selected_option'].map(age_midpoints)
years_midpoints = {'A': 0.5, 'B': 2, 'C': 4, 'D': 7.5, 'E': 12}
years_raw = answers[answers['question_id'] == 2]['selected_option'].map(years_midpoints)

print(f'\n背景画像摘要:')
print(f'  估算平均年龄: {age_raw.mean():.1f} 岁')
print(f'  估算平均来美年数: {years_raw.mean():.1f} 年')
print(f'  性别比例: {dict(gender_counts)}')
print(f'  最多城市级别: {city_counts.idxmax()} ({city_counts.max()}人)')

In [ ]:
# ---------- 4.5 各象限的背景画像交叉分析 ----------
# 合并背景信息到 session 级别
bg_answers = answers[answers['question_id'].isin([0,1,2,3])][['session_id','question_id','selected_option']].copy()
bg_pivot = bg_answers.pivot(index='session_id', columns='question_id', values='selected_option')
bg_pivot.columns = ['gender', 'city', 'years', 'age']
bg_pivot['gender_label'] = bg_pivot['gender'].map(OPTION_LABELS[0])
bg_pivot['city_label'] = bg_pivot['city'].map(OPTION_LABELS[1])
bg_pivot['years_label'] = bg_pivot['years'].map(OPTION_LABELS[2])
bg_pivot['age_label'] = bg_pivot['age'].map(OPTION_LABELS[3])

bg_full = bg_pivot.merge(sessions[['id','quadrant','quadrant_label']], left_index=True, right_on='id')

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# 性别 × 象限
ax = axes[0]
ct = pd.crosstab(bg_full['quadrant_label'], bg_full['gender_label'], normalize='index') * 100
ct = ct.reindex(columns=[c for c in ['女性','男性','非二元/不透露'] if c in ct.columns])
ct.plot(kind='barh', stacked=True, ax=ax, color=['#ff6b9d','#4ecdc4','#ffe66d'][:len(ct.columns)])
ax.set_xlabel('百分比 (%)', fontproperties=zh_font)
ax.set_title('各象限性别构成', fontproperties=zh_font, fontsize=13, fontweight='bold')
ax.set_yticklabels(ax.get_yticklabels(), fontproperties=zh_font)
ax.legend(prop=zh_font, loc='lower right')

# 年龄 × 象限
ax = axes[1]
ct2 = pd.crosstab(bg_full['quadrant_label'], bg_full['age_label'], normalize='index') * 100
ct2 = ct2.reindex(columns=[c for c in age_order if c in ct2.columns])
ct2.plot(kind='barh', stacked=True, ax=ax, colormap='YlGnBu')
ax.set_xlabel('百分比 (%)', fontproperties=zh_font)
ax.set_title('各象限年龄构成', fontproperties=zh_font, fontsize=13, fontweight='bold')
ax.set_yticklabels(ax.get_yticklabels(), fontproperties=zh_font)
ax.legend(prop=zh_font, loc='lower right', fontsize=8)

# 来美年数 × 象限
ax = axes[2]
ct3 = pd.crosstab(bg_full['quadrant_label'], bg_full['years_label'], normalize='index') * 100
ct3 = ct3.reindex(columns=[c for c in years_order if c in ct3.columns])
ct3.plot(kind='barh', stacked=True, ax=ax, colormap='Oranges')
ax.set_xlabel('百分比 (%)', fontproperties=zh_font)
ax.set_title('各象限来美年数构成', fontproperties=zh_font, fontsize=13, fontweight='bold')
ax.set_yticklabels(ax.get_yticklabels(), fontproperties=zh_font)
ax.legend(prop=zh_font, loc='lower right', fontsize=8)

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/03_demographics_by_quadrant.png', bbox_inches='tight')
plt.show()
print('各象限背景画像交叉分析完成')

---
## 5. 关注点分析：权重偏好与核心诉求

权重反映了用户对每个维度的**主观重要性**。权重 1=不在乎，5=核心诉求。  
基础信息题（Q0-3）默认 weight=3，此处仅分析 Q4-Q28 的用户自选权重。

In [ ]:
# 排除基础信息题（auto_weight=3）
weighted_ans = answers[answers['question_id'] >= 4].copy()

# 各题平均权重
avg_weight = weighted_ans.groupby('question_id')['weight'].mean().sort_values(ascending=False)
avg_weight_df = avg_weight.reset_index()
avg_weight_df['title'] = avg_weight_df['question_id'].map(QUESTION_TITLES)

fig, ax = plt.subplots(figsize=(12, 10))
colors_bar = ['#667eea' if w >= avg_weight.mean() else '#a0aec0' for w in avg_weight_df['weight']]
bars = ax.barh(range(len(avg_weight_df)), avg_weight_df['weight'].values, color=colors_bar, edgecolor='white')
ax.set_yticks(range(len(avg_weight_df)))
ax.set_yticklabels([f'Q{row.question_id} {row.title}' for _, row in avg_weight_df.iterrows()], 
                    fontproperties=zh_font, fontsize=10)
ax.axvline(x=avg_weight.mean(), color='red', linestyle='--', alpha=0.7, label=f'总平均 {avg_weight.mean():.2f}')
ax.axvline(x=3.0, color='gray', linestyle=':', alpha=0.5, label='默认值 3.0')

for bar, val in zip(bars, avg_weight_df['weight'].values):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2, 
            f'{val:.2f}', va='center', fontsize=9)

ax.set_xlabel('平均权重 (1=不在乎, 5=核心诉求)', fontproperties=zh_font, fontsize=12)
ax.set_title('各维度平均权重排名 — 大家最在乎什么？', fontproperties=zh_font, fontsize=14, fontweight='bold')
ax.legend(prop=zh_font, fontsize=10)
ax.set_xlim(1, 5)
ax.invert_yaxis()

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/04_weight_ranking.png', bbox_inches='tight')
plt.show()

print('\nTOP 5 最受关注维度:')
for i, (_, row) in enumerate(avg_weight_df.head(5).iterrows()):
    print(f'  {i+1}. Q{row.question_id} {row.title}: 平均权重 {row.weight:.2f}')

print('\nBOTTOM 5 最不受关注维度:')
for i, (_, row) in enumerate(avg_weight_df.tail(5).iterrows()):
    print(f'  {i+1}. Q{row.question_id} {row.title}: 平均权重 {row.weight:.2f}')

In [ ]:
# ---------- 5.2 高权重(4-5)选择比例 — 谁被认为是"核心诉求" ----------
high_weight = weighted_ans[weighted_ans['weight'] >= 4]
high_pct = high_weight.groupby('question_id').size() / weighted_ans.groupby('question_id').size() * 100
high_pct = high_pct.sort_values(ascending=False)
high_pct_df = high_pct.reset_index()
high_pct_df.columns = ['question_id', 'high_pct']
high_pct_df['title'] = high_pct_df['question_id'].map(QUESTION_TITLES)

fig, ax = plt.subplots(figsize=(12, 10))
colors_hp = ['#f5576c' if p >= high_pct.mean() else '#fed7d7' for p in high_pct_df['high_pct']]
bars = ax.barh(range(len(high_pct_df)), high_pct_df['high_pct'].values, color=colors_hp, edgecolor='white')
ax.set_yticks(range(len(high_pct_df)))
ax.set_yticklabels([f'Q{row.question_id} {row.title}' for _, row in high_pct_df.iterrows()], 
                    fontproperties=zh_font, fontsize=10)
ax.axvline(x=high_pct.mean(), color='red', linestyle='--', alpha=0.7, label=f'平均 {high_pct.mean():.1f}%')

for bar, val in zip(bars, high_pct_df['high_pct'].values):
    ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, 
            f'{val:.1f}%', va='center', fontsize=9)

ax.set_xlabel('选择权重4-5的用户比例 (%)', fontproperties=zh_font, fontsize=12)
ax.set_title('"核心诉求"比例排名 — 多少人认为该维度极其重要？', fontproperties=zh_font, fontsize=14, fontweight='bold')
ax.legend(prop=zh_font)
ax.invert_yaxis()

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/05_high_weight_pct.png', bbox_inches='tight')
plt.show()

In [ ]:
# ---------- 5.3 按类别聚合的平均权重 ----------
cat_weights = []
for cat, qids in CATEGORIES.items():
    non_auto = [q for q in qids if q >= 4]  # 排除 auto_weight
    if non_auto:
        w = weighted_ans[weighted_ans['question_id'].isin(non_auto)]['weight'].mean()
        cat_weights.append({'category': cat, 'avg_weight': w, 'n_questions': len(non_auto)})

cat_df = pd.DataFrame(cat_weights)

fig, ax = plt.subplots(figsize=(10, 5))
cat_colors = ['#667eea', '#43e97b', '#f5576c']
bars = ax.bar(cat_df['category'], cat_df['avg_weight'], color=cat_colors, edgecolor='white', width=0.5)
for bar, val in zip(bars, cat_df['avg_weight']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{val:.2f}', ha='center', fontsize=12, fontweight='bold')
ax.set_xticklabels(cat_df['category'], fontproperties=zh_font, fontsize=11)
ax.set_ylabel('平均权重', fontproperties=zh_font, fontsize=12)
ax.set_title('各维度类别的平均关注度', fontproperties=zh_font, fontsize=14, fontweight='bold')
ax.set_ylim(1, 4.5)
ax.axhline(y=3.0, color='gray', linestyle=':', alpha=0.5)

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/06_category_weight.png', bbox_inches='tight')
plt.show()

print('按类别的平均关注度:')
for _, row in cat_df.iterrows():
    print(f'  {row.category} ({row.n_questions}题): {row.avg_weight:.2f}')

---
## 6. 维度差异化分析：方差最大的题目

方差越大，表明用户群体在该维度上分歧最大（即最"撕裂"的议题）。  
这里分析两种方差：选项选择分布的方差、以及加权得分 (US-CN) 的方差。

In [ ]:
# ---------- 6.1 US-CN 加权差值的方差 ----------
answers['us_cn_diff'] = answers['us_weighted'] - answers['cn_weighted']

variance_by_q = answers.groupby('question_id')['us_cn_diff'].var().sort_values(ascending=False)
var_df = variance_by_q.reset_index()
var_df.columns = ['question_id', 'variance']
var_df['title'] = var_df['question_id'].map(QUESTION_TITLES)
var_df['std'] = np.sqrt(var_df['variance'])

fig, ax = plt.subplots(figsize=(12, 10))
# 颜色梯度: 方差大的红，小的蓝
norm = plt.Normalize(var_df['variance'].min(), var_df['variance'].max())
cmap = plt.cm.RdYlGn_r
bar_colors = [cmap(norm(v)) for v in var_df['variance']]

bars = ax.barh(range(len(var_df)), var_df['variance'].values, color=bar_colors, edgecolor='white')
ax.set_yticks(range(len(var_df)))
ax.set_yticklabels([f'Q{row.question_id} {row.title}' for _, row in var_df.iterrows()], 
                    fontproperties=zh_font, fontsize=10)

for bar, (_, row) in zip(bars, var_df.iterrows()):
    ax.text(bar.get_width() + 0.2, bar.get_y() + bar.get_height()/2, 
            f'{row.variance:.1f} (σ={row["std"]:.1f})', va='center', fontsize=9)

ax.set_xlabel('(US得分 - CN得分) 的方差', fontproperties=zh_font, fontsize=12)
ax.set_title('各维度"分歧度"排名 — 方差越大 = 用户群体分歧越大', fontproperties=zh_font, fontsize=14, fontweight='bold')
ax.invert_yaxis()

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/07_variance_ranking.png', bbox_inches='tight')
plt.show()

print('\nTOP 5 分歧最大的维度（方差最高）:')
for i, (_, row) in enumerate(var_df.head(5).iterrows()):
    print(f'  {i+1}. Q{row.question_id} {row.title}: 方差={row.variance:.1f}, 标准差={row["std"]:.1f}')

print('\nTOP 5 共识最强的维度（方差最低）:')
for i, (_, row) in enumerate(var_df.tail(5).iterrows()):
    print(f'  {i+1}. Q{row.question_id} {row.title}: 方差={row.variance:.1f}, 标准差={row["std"]:.1f}')

In [ ]:
# ---------- 6.2 TOP5 最分裂题目的选项分布 ----------
top5_var_qids = var_df.head(5)['question_id'].tolist()

fig, axes = plt.subplots(1, 5, figsize=(20, 5))

for idx, qid in enumerate(top5_var_qids):
    ax = axes[idx]
    q_data = answers[answers['question_id'] == qid]
    title = QUESTION_TITLES[qid]
    
    if qid in range(4, 13):  # dual_select, 展示 US-CN diff 分布
        ax.hist(q_data['us_cn_diff'], bins=20, color='#667eea', alpha=0.7, edgecolor='white')
        ax.axvline(x=0, color='red', linestyle='--', alpha=0.7)
        ax.set_xlabel('US-CN差值', fontproperties=zh_font, fontsize=9)
    else:  # single_choice, 展示选项分布
        opt_counts = q_data['selected_option'].value_counts().sort_index()
        ax.bar(opt_counts.index, opt_counts.values, color='#667eea', edgecolor='white')
        ax.set_xlabel('选项', fontproperties=zh_font, fontsize=9)
    
    ax.set_title(f'Q{qid} {title}', fontproperties=zh_font, fontsize=10, fontweight='bold')
    ax.set_ylabel('人数', fontproperties=zh_font, fontsize=9)

plt.suptitle('TOP 5 分歧最大维度的答案分布', fontproperties=zh_font, fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(f'{IMG_DIR}/08_top5_variance_detail.png', bbox_inches='tight')
plt.show()

---
## 7. 分群对比：留美派 vs 回国派的核心差异

将"坚定留美派"(us_high_cn_low) 与 "果断回国派"(us_low_cn_high) 两个最鲜明的群体进行对比，  
找出**哪些维度对去留决策贡献最大**。

In [ ]:
# ---------- 7.1 留美派 vs 回国派: 各维度平均加权得分差 ----------
us_group = answers[answers['quadrant'] == 'us_high_cn_low']
cn_group = answers[answers['quadrant'] == 'us_low_cn_high']

# 对每题，计算两组的平均 us_weighted 和 cn_weighted
us_means = us_group.groupby('question_id')[['us_weighted', 'cn_weighted']].mean()
cn_means = cn_group.groupby('question_id')[['us_weighted', 'cn_weighted']].mean()

# 净得分差 = (US组的US偏好 - CN组的US偏好) - (US组的CN偏好 - CN组的CN偏好)
# 简化为：两组在 us_weighted - cn_weighted 上的差距
us_net = (us_means['us_weighted'] - us_means['cn_weighted'])  # 留美派每题的 US偏好净值
cn_net = (cn_means['us_weighted'] - cn_means['cn_weighted'])  # 回国派每题的 US偏好净值
gap = (us_net - cn_net).sort_values(ascending=False)  # 差距越大 = 该维度对去留分化贡献越大

gap_df = gap.reset_index()
gap_df.columns = ['question_id', 'gap']
gap_df['title'] = gap_df['question_id'].map(QUESTION_TITLES)

fig, ax = plt.subplots(figsize=(12, 10))
bar_colors = ['#667eea' if g > 0 else '#f5576c' for g in gap_df['gap']]
bars = ax.barh(range(len(gap_df)), gap_df['gap'].values, color=bar_colors, edgecolor='white')
ax.set_yticks(range(len(gap_df)))
ax.set_yticklabels([f'Q{row.question_id} {row.title}' for _, row in gap_df.iterrows()], 
                    fontproperties=zh_font, fontsize=10)
ax.axvline(x=0, color='black', linewidth=0.8)

for bar, val in zip(bars, gap_df['gap'].values):
    offset = 0.05 if val >= 0 else -0.05
    ha = 'left' if val >= 0 else 'right'
    ax.text(bar.get_width() + offset, bar.get_y() + bar.get_height()/2, 
            f'{val:.2f}', va='center', ha=ha, fontsize=9)

ax.set_xlabel('留美派 vs 回国派 净偏好差距（正=留美派优势维度）', fontproperties=zh_font, fontsize=11)
ax.set_title('哪些维度最能区分"留美"和"回国"人群？', fontproperties=zh_font, fontsize=14, fontweight='bold')
ax.invert_yaxis()

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/09_us_vs_cn_gap.png', bbox_inches='tight')
plt.show()

print('\n留美派最大优势维度 (这些维度把人"拉向"美国):')
for i, (_, row) in enumerate(gap_df.head(5).iterrows()):
    print(f'  {i+1}. Q{row.question_id} {row.title}: gap={row.gap:.2f}')

print('\n回国派最大优势维度 (这些维度把人"拉回"中国):')
for i, (_, row) in enumerate(gap_df.tail(5).iloc[::-1].iterrows()):
    print(f'  {i+1}. Q{row.question_id} {row.title}: gap={row.gap:.2f}')

In [ ]:
# ---------- 7.2 留美派 vs 回国派 各维度雷达图对比 ----------
# 排除基础信息，只比较 Q4-Q28 的25题
compare_qids = list(range(4, 29))
compare_titles = [QUESTION_TITLES[q] for q in compare_qids]

us_group_avg = us_group[us_group['question_id'].isin(compare_qids)].groupby('question_id')['weight'].mean()
cn_group_avg = cn_group[cn_group['question_id'].isin(compare_qids)].groupby('question_id')['weight'].mean()

us_vals = [us_group_avg.get(q, 3) for q in compare_qids]
cn_vals = [cn_group_avg.get(q, 3) for q in compare_qids]

# 雷达图
angles = np.linspace(0, 2 * np.pi, len(compare_qids), endpoint=False).tolist()
us_vals_r = us_vals + [us_vals[0]]
cn_vals_r = cn_vals + [cn_vals[0]]
angles_r = angles + [angles[0]]

fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(polar=True))
ax.plot(angles_r, us_vals_r, 'o-', color='#667eea', linewidth=2, label='坚定留美派', markersize=4)
ax.fill(angles_r, us_vals_r, alpha=0.15, color='#667eea')
ax.plot(angles_r, cn_vals_r, 'o-', color='#f5576c', linewidth=2, label='果断回国派', markersize=4)
ax.fill(angles_r, cn_vals_r, alpha=0.15, color='#f5576c')

ax.set_xticks(angles)
ax.set_xticklabels(compare_titles, fontproperties=zh_font, fontsize=8)
ax.set_ylim(1, 5)
ax.set_title('留美派 vs 回国派 — 各维度关注度(权重)对比雷达图', 
             fontproperties=zh_font, fontsize=14, fontweight='bold', pad=30)
ax.legend(prop=zh_font, loc='upper right', bbox_to_anchor=(1.2, 1.1), fontsize=11)

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/10_radar_us_vs_cn.png', bbox_inches='tight')
plt.show()
print('雷达图展示了两组人群在各维度上的"关注度"差异。距中心越远 = 越在乎该维度。')

In [ ]:
# ---------- 7.3 四象限人群各维度的平均得分热力图 ----------
# 计算各象限各题的平均 (us_weighted - cn_weighted)
heatmap_data = answers.groupby(['quadrant_label', 'question_id'])['us_cn_diff'].mean().unstack()
heatmap_data.columns = [f'Q{c} {QUESTION_TITLES[c]}' for c in heatmap_data.columns]

fig, ax = plt.subplots(figsize=(20, 5))
sns.heatmap(heatmap_data, cmap='RdBu', center=0, annot=True, fmt='.1f', 
            linewidths=0.5, ax=ax, cbar_kws={'label': 'US偏好 - CN偏好'})
ax.set_xticklabels(ax.get_xticklabels(), fontproperties=zh_font, fontsize=8, rotation=45, ha='right')
ax.set_yticklabels(ax.get_yticklabels(), fontproperties=zh_font, fontsize=10)
ax.set_title('四象限人群 × 29维度 得分偏好热力图（正=偏美国，负=偏中国）', 
             fontproperties=zh_font, fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/11_heatmap_quadrant_questions.png', bbox_inches='tight')
plt.show()
print('热力图中：蓝色=偏美国，红色=偏中国。颜色越深=偏好越强。')

---
## 8. 美国得分 vs 中国得分 散点象限图

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

for quad in labels_order:
    subset = sessions[sessions['quadrant'] == quad]
    ax.scatter(subset['cn_total_score'], subset['us_total_score'], 
               c=QUADRANT_COLORS[quad], label=QUADRANT_LABELS[quad],
               alpha=0.5, s=30, edgecolors='white', linewidth=0.3)

# 阈值线
ax.axhline(y=130.5, color='gray', linestyle='--', alpha=0.6)
ax.axvline(x=130.5, color='gray', linestyle='--', alpha=0.6)

# 象限标签
text_props = dict(fontproperties=zh_font, fontsize=12, alpha=0.4, fontweight='bold')
ax.text(0.75, 0.75, '跨国撕裂型', transform=ax.transAxes, ha='center', **text_props)
ax.text(0.25, 0.75, '坚定留美派', transform=ax.transAxes, ha='center', **text_props)
ax.text(0.75, 0.25, '果断回国派', transform=ax.transAxes, ha='center', **text_props)
ax.text(0.25, 0.25, '两难探索型', transform=ax.transAxes, ha='center', **text_props)

ax.set_xlabel('中国吸引力总分 →', fontproperties=zh_font, fontsize=13)
ax.set_ylabel('美国吸引力总分 →', fontproperties=zh_font, fontsize=13)
ax.set_title(f'留学生去留决策 散点象限图 (N={len(sessions)})', 
             fontproperties=zh_font, fontsize=15, fontweight='bold')
ax.legend(prop=zh_font, fontsize=11, loc='upper left')

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/12_scatter_quadrant.png', bbox_inches='tight')
plt.show()

# 2D KDE 密度图
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(sessions['cn_total_score'], sessions['us_total_score'], 
           alpha=0.15, s=15, c='#667eea')
try:
    sns.kdeplot(x=sessions['cn_total_score'], y=sessions['us_total_score'],
                levels=8, color='#764ba2', linewidths=1.5, ax=ax)
except:
    pass
ax.axhline(y=130.5, color='gray', linestyle='--', alpha=0.6)
ax.axvline(x=130.5, color='gray', linestyle='--', alpha=0.6)
ax.set_xlabel('中国吸引力总分 →', fontproperties=zh_font, fontsize=13)
ax.set_ylabel('美国吸引力总分 →', fontproperties=zh_font, fontsize=13)
ax.set_title(f'得分密度分布 (KDE等高线)', fontproperties=zh_font, fontsize=15, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/13_scatter_kde.png', bbox_inches='tight')
plt.show()

In [ ]:
# ---------- 8.2 得分分布直方图 ----------
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.hist(sessions['us_total_score'], bins=40, color='#667eea', alpha=0.7, edgecolor='white')
ax1.axvline(x=130.5, color='red', linestyle='--', label=f'阈值 130.5')
ax1.axvline(x=sessions['us_total_score'].mean(), color='orange', linestyle='-', 
            label=f'均值 {sessions["us_total_score"].mean():.1f}')
ax1.set_xlabel('美国吸引力总分', fontproperties=zh_font, fontsize=12)
ax1.set_ylabel('人数', fontproperties=zh_font, fontsize=12)
ax1.set_title('美国吸引力得分分布', fontproperties=zh_font, fontsize=14, fontweight='bold')
ax1.legend(prop=zh_font)

ax2.hist(sessions['cn_total_score'], bins=40, color='#f5576c', alpha=0.7, edgecolor='white')
ax2.axvline(x=130.5, color='red', linestyle='--', label=f'阈值 130.5')
ax2.axvline(x=sessions['cn_total_score'].mean(), color='orange', linestyle='-', 
            label=f'均值 {sessions["cn_total_score"].mean():.1f}')
ax2.set_xlabel('中国吸引力总分', fontproperties=zh_font, fontsize=12)
ax2.set_ylabel('人数', fontproperties=zh_font, fontsize=12)
ax2.set_title('中国吸引力得分分布', fontproperties=zh_font, fontsize=14, fontweight='bold')
ax2.legend(prop=zh_font)

plt.tight_layout()
plt.savefig(f'{IMG_DIR}/14_score_histograms.png', bbox_inches='tight')
plt.show()

---
## 9. 总结

In [ ]:
# ---------- 生成文字总结 ----------
total = len(sessions)
q_counts = sessions['quadrant'].value_counts()

top_weight_q = avg_weight_df.iloc[0]
top_var_q = var_df.iloc[0]
top_gap_us = gap_df.iloc[0]
top_gap_cn = gap_df.iloc[-1]

summary = f"""
{'='*60}
  留学生去留决策量表 V2 — 数据分析总结报告
{'='*60}

一、基本数据
  - 总填写人数: {total} 人
  - 平均美国吸引力得分: {sessions['us_total_score'].mean():.1f} (阈值 130.5)
  - 平均中国吸引力得分: {sessions['cn_total_score'].mean():.1f} (阈值 130.5)
  - 估算平均年龄: {age_raw.mean():.1f} 岁
  - 估算平均来美年数: {years_raw.mean():.1f} 年

二、象限分布
  - 坚定留美派: {q_counts.get('us_high_cn_low',0)} 人 ({q_counts.get('us_high_cn_low',0)/total*100:.1f}%)
  - 果断回国派: {q_counts.get('us_low_cn_high',0)} 人 ({q_counts.get('us_low_cn_high',0)/total*100:.1f}%)
  - 跨国撕裂型: {q_counts.get('us_high_cn_high',0)} 人 ({q_counts.get('us_high_cn_high',0)/total*100:.1f}%)
  - 两难探索型: {q_counts.get('us_low_cn_low',0)} 人 ({q_counts.get('us_low_cn_low',0)/total*100:.1f}%)

三、核心发现
  1. 最受关注维度: Q{top_weight_q.question_id} {top_weight_q.title} (平均权重 {top_weight_q.weight:.2f})
  2. 分歧最大维度: Q{top_var_q.question_id} {top_var_q.title} (方差 {top_var_q.variance:.1f})
  3. 最拉向美国的维度: Q{top_gap_us.question_id} {top_gap_us.title} (gap={top_gap_us.gap:.2f})
  4. 最拉回中国的维度: Q{top_gap_cn.question_id} {top_gap_cn.title} (gap={top_gap_cn.gap:.2f})

四、关键洞察
  - 超过半数用户落入"坚定留美派"象限，反映样本总体偏好留美
  - 身份状态(签证/绿卡)、薪资购买力 是最能区分留美 vs 回国的硬性指标
  - 家庭照料义务、国内人脉 是最强的"回国拉力"
  - 权重分析显示用户普遍最在意"硬性条件"(职业/财务), 而非"软性体验"(文娱/旅行)

{'='*60}
  报告生成完毕 | 图表已保存至 analysis/charts/ 目录
{'='*60}
"""

print(summary)

# 保存文字总结
with open(f'{IMG_DIR}/../summary.txt', 'w', encoding='utf-8') as f:
    f.write(summary)
print(f'\n文字总结已保存至 analysis/summary.txt')